# 4.8 Grouping Data and Aggregating Variables

## Contents

1. Importing Libraries
2. Importing Data
3. Aggregating Data
4. Exporting Data

## 01. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

## 02. Importing Data

In [2]:
path = r'C:\Users\kensz\Documents\orders_products_merged_with_new_variables'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'orders_products_merged_with_new_variables.pkl'))

In [4]:
ords_prods_merge.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours,busiest_period_of_day
0,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly Busy,Least Busy Days,Average Orders,Average Orders
1,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly Busy,Least Busy Days,Most Orders,Most Orders
2,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least Busy,Least Busy Days,Average Orders,Average Orders
3,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least Busy,Least Busy Days,Most Orders,Most Orders
4,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly Busy,Regularly Busy,Average Orders,Average Orders


In [5]:
ords_prods_merge.shape

(30328763, 19)

## 03. Aggregating Columns

In [6]:
#aggregated mean for order_number grouped by department id

ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,16.559358
2,18.413176
3,18.279600
4,18.915890
5,16.497751
6,17.609390
7,18.303975
8,16.383301


Compared to running the same process on the subset of data during the task, this data is significantly different. The first noticeable difference is that theis returns data for all of the departments, while the subset of the first 1,000,000 only included five departments. Additionally the means of columns included in the subset have changed, presumeably because new values are being included in the calculation for the department compared to the subset. 

### Spending Habits Based off Loyalty Status

In [7]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty flag'] = 'Loyal Customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty flag'] = 'Regular Customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 10, 'loyalty flag')] ='New Customer'

In [9]:
ords_prods_merge['loyalty flag'].value_counts(dropna = False)

Regular Customer    15081691
Loyal Customer      10095381
New Customer         5151691
Name: loyalty flag, dtype: int64

In [13]:
ords_prods_merge.groupby('loyalty flag').agg({'prices':['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty flag                             
Loyal Customer    10.402162  1.0  99999.0
New Customer      13.396333  1.0  99999.0
Regular Customer  12.546842  1.0  99999.0

There is no signifigant difference between the spending habits of customers based off thier loyalty. 

### Targeting Based on Spending Habits

In [14]:
ords_prods_merge.groupby('user_id').agg({'prices': ['mean']})

,prices
,mean
user_id,
1,6.372222
2,7.583516
3,8.302564
4,7.607143
5,9.365385
...,...
206205,9.066667
206206,7.601068


In [15]:
ords_prods_merge['average_prices'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
ords_prods_merge.head(25)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours,busiest_period_of_day,max_order,loyalty flag,average_prices
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly Busy,Least Busy Days,Average Orders,Average Orders,10,New Customer,6.372222
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly Busy,Least Busy Days,Most Orders,Most Orders,10,New Customer,6.372222
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least Busy,Least Busy Days,Average Orders,Average Orders,10,New Customer,6.372222
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least Busy,Least Busy Days,Most Orders,Most Orders,10,New Customer,6.372222
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly Busy,Regularly Busy,Average Orders,Average Orders,10,New Customer,6.372222
5,550135,1,7,1,9,20.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly Busy,Busiest Days,Average Orders,Average Orders,10,New Customer,6.372222
6,3108588,1,8,1,14,14.0,196,2,1,Soda,...,9.0,both,Mid-range product,Regularly Busy,Busiest Days,Most Orders,Most Orders,10,New Customer,6.372222
7,2295261,1,9,1,16,0.0,196,4,1,Soda,...,9.0,both,Mid-range product,Regularly Busy,Busiest Days,Average Orders,Average Orders,10,New Customer,6.372222
8,2550362,1,10,4,8,30.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least Busy,Least Busy Days,Average Orders,Average Orders,10,New Customer,6.372222
9,2968173,15,15,1,9,7.0,196,2,0,Soda,...,9.0,both,Mid-range product,Regularly Busy,Busiest Days,Average Orders,Average Orders,22,Regular Customer,3.994118


In [18]:
ords_prods_merge.loc[ords_prods_merge['average_prices'] < 10, 'Spending_Flag'] = 'Low Spender'
ords_prods_merge.loc[ords_prods_merge['average_prices'] >= 10, 'Spending_Flag'] = 'High Spender'ords

In [20]:
ords_prods_merge['Spending_Flag'].value_counts(dropna = False)

Low Spender     29730505
High Spender      598258
Name: Spending_Flag, dtype: int64

### Order Frequnecy

In [21]:
ords_prods_merge['median_order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [22]:
ords_prods_merge.head(25)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_hours,busiest_period_of_day,max_order,loyalty flag,average_prices,Spending_Flag,median_order_frequency
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly Busy,Least Busy Days,Average Orders,Average Orders,10,New Customer,6.372222,Low Spender,20.5
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly Busy,Least Busy Days,Most Orders,Most Orders,10,New Customer,6.372222,Low Spender,20.5
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least Busy,Least Busy Days,Average Orders,Average Orders,10,New Customer,6.372222,Low Spender,20.5
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least Busy,Least Busy Days,Most Orders,Most Orders,10,New Customer,6.372222,Low Spender,20.5
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Mid-range product,Regularly Busy,Regularly Busy,Average Orders,Average Orders,10,New Customer,6.372222,Low Spender,20.5
5,550135,1,7,1,9,20.0,196,1,1,Soda,...,Mid-range product,Regularly Busy,Busiest Days,Average Orders,Average Orders,10,New Customer,6.372222,Low Spender,20.5
6,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Mid-range product,Regularly Busy,Busiest Days,Most Orders,Most Orders,10,New Customer,6.372222,Low Spender,20.5
7,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Mid-range product,Regularly Busy,Busiest Days,Average Orders,Average Orders,10,New Customer,6.372222,Low Spender,20.5
8,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Mid-range product,Least Busy,Least Busy Days,Average Orders,Average Orders,10,New Customer,6.372222,Low Spender,20.5
9,2968173,15,15,1,9,7.0,196,2,0,Soda,...,Mid-range product,Regularly Busy,Busiest Days,Average Orders,Average Orders,22,Regular Customer,3.994118,Low Spender,10.0


In [31]:
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] > 20, 'frequency_tag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_order_frequency'] > 10) & (ords_prods_merge['median_order_frequency'] <= 20), 'frequency_tag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] <= 10, 'frequency_tag'] = 'Frequent customer'

In [32]:
ords_prods_merge['frequency_tag'].value_counts(dropna = False)

Frequent customer        20675290
Regular customer          6594542
Non-frequent customer     3058931
Name: frequency_tag, dtype: int64

## 04. Exporting Data

In [35]:
path2 = r'C:\Users\kensz\OneDrive\Desktop\CF Data Analytics Course\Achievement 4\11-2022 Instacart basket Analysis'

In [37]:
ords_prods_merge.to_pickle(os.path.join(path2, '02 Data', 'Prepared Data', 'orders_products_with_aggregates.pkl'))